# **From Tweets to Hope: A Data-Driven Approach to Supporting the Needy in America**






In [ ]:
# Installs the Python package "vaderSentiment" via pip, a package installer for Python.
!pip install vaderSentiment

In [ ]:
# Installs the Python package "flair" via pip, a package installer for Python.
!pip install flair

In [ ]:
# Installs the Python package "chart_studio" via pip, a package installer for Python.
!pip install chart_studio

In [ ]:
# Installs the Python package "plotly" via pip, a package installer for Python.
!pip install plotly

In [ ]:
#  Installs the Python package "geopandas" via pip, a package installer for Python.
!pip install geopandas

In [ ]:
import tweepy
import re
import pandas as pd
import spacy
import nltk
import flair
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.offline as po
import plotly.graph_objs as pg

from textblob import TextBlob
from flair.models import TextClassifier
from flair.data import Sentence
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk import FreqDist
from collections import defaultdict
from google.colab import files

nlp = spacy.load("en_core_web_sm")

# A dataset used for tokenizing text
nltk.download('punkt')

# A dataset used for part-of-speech tagging
nltk.download('averaged_perceptron_tagger')

# Downloads the 'stopwords' corpus, which is a list of commonly used words (such as "the", "and", etc.) that are usually removed from text during pre-processing
nltk.download('stopwords')
nltk_stops = nltk.corpus.stopwords.words('english')

# Downloads the 'wordnet' and 'omw-1.4' corpus, which is used for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')
wnl = nltk.WordNetLemmatizer()

In [7]:
CONSUMER_KEY = # Consumer Key
CONSUMER_SECRET = # Consumer Secret
OAUTH_TOKEN = # Oauth Token
OAUTH_TOKEN_SECRET = # Oauth Token Secret


# Authenticate Twitter API credentials
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)

# **Data Extraction**

**1) Needy**

In [ ]:
# Collect tweets
needyTweets = {}
geocode = '"39.8,-95.583068847656,2500km"'
# Loop through tweets containing specific keywords and within a specific geographic area
for tweet in tweepy.Cursor(api.search_tweets, q = "UrgentHelp OR helpneeded OR Needmoneyforsurgery\
                                                   OR DonateForACause OR FeedTheHungry OR Homelessoutreach\
                                                   OR FeedTheHungry OR Homelessoutreach\
                                                   OR EarthquakeRelief OR CycloneRelief OR GoFundMe\
                                                   OR gofundmedonations OR 4charity -filter:retweets",
                           geocode = geocode,
                           lang = 'en',
                           tweet_mode = 'extended',
                           count = 100).items(10000):
  # Removing retweets
  if not tweet.retweeted:
    # Create dictionary of tweet details and store in 'needyTweets' dictionary using tweet id as key
    tweetDetails = {
        'name': tweet.user.screen_name,
        'tweetID': tweet.id,
        'location': tweet.user.location,
        'full_text': tweet.full_text
    }
    needyTweets[tweet.id] = tweetDetails

**2) Donor**

In [ ]:
# Collect tweets
donorTweets = {}
# Represents Latitute and Longitue along with 2500 km Radius.
geocode = '"39.8,-95.583068847656,2500km"'
# Iterate through the Twitter API search results
for tweet in tweepy.Cursor(api.search_tweets, q = "charitysector OR mainedayofgiving OR CharitableGiving\
                                                   OR CommunityService OR MakingADifference OR givingbacktothecommunity\
                                                   OR CharityWork OR Volunteering OR HumanityFirst OR DigitalFundraising\
                                                   OR GivingBack OR NGO OR donationsappreciated OR blooddonor\
                                                   OR plasmadonor OR organdonor -filter:retweets",
                           geocode = geocode,
                           lang = 'en',
                           tweet_mode = 'extended',
                           count=100).items(10000):

  # Removing retweets
  if not tweet.retweeted:
    # Extract tweet details and add to donorTweets dictionary
    tweetDetails = {
        'name': tweet.user.screen_name,
        'tweetID': tweet.id,
        'location': tweet.user.location,
        'full_text': tweet.full_text
    }
    # Add the tweet to the donorTweets dictionary with tweet ID as key
    donorTweets[tweet.id] = tweetDetails

# **Data Preprocessing**

In [ ]:
# Actual Data Pre-processing is done here.
def preprocessText(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    # Consider strings containing only alphabets and numbers
    words = [w for w in tokens if w.isalnum()]
    # Remove stop words
    words = [word for word in words if word not in nltk_stops]
    # Lemmatize words
    words = [wnl.lemmatize(word) for word in words]
    # Return processed text
    return words

**1) Needy**

In [ ]:
# Pre-process and Clean-up tweets
# A dictionary to store cleaned tweets
cleaned_needy_tweets = {}
wordListNeedy = []
for tweetId, tweetDetails in needyTweets.items():
    # remove URLs
    cleaned_tweet = re.sub(r'http\S+', '', tweetDetails['full_text'])
    # remove user mentions
    cleaned_tweet = re.sub(r'@\S+', '', cleaned_tweet)
    # remove hashtags
    cleaned_tweet = re.sub(r'#\S+', '', cleaned_tweet)
    # remove leading/trailing white space
    cleaned_tweet = cleaned_tweet.strip()
    # only keep non-empty tweets
    if cleaned_tweet:
        cleaned_needy_tweets[tweetId] = cleaned_tweet
        wordListNeedy.extend(preprocessText(cleaned_tweet))

**2) Donor**

In [ ]:
# Pre-process and Clean-up tweets
cleaned_donor_tweets = {}
wordListDonor = []
for tweetId, tweetDetails in donorTweets.items():
    # remove URLs
    cleaned_tweet = re.sub(r'http\S+', '', tweetDetails['full_text'])
    # remove user mentions
    cleaned_tweet = re.sub(r'@\S+', '', cleaned_tweet)
    # remove hashtags
    cleaned_tweet = re.sub(r'#\S+', '', cleaned_tweet)
    # remove leading/trailing white space
    cleaned_tweet = cleaned_tweet.strip()
    # only keep non-empty tweets
    if cleaned_tweet:
        cleaned_donor_tweets[tweetId] = cleaned_tweet
        wordListDonor.extend(preprocessText(cleaned_tweet))

# **Frequency Distribution**

**1) Needy**

In [ ]:
# Calculate the frequency distribution of words in the cleaned tweets
frequentWordListNeedy = nltk.FreqDist(wordListNeedy)
# Get the 30 most common words from the frequency distribution
importantWordListNeedy = frequentWordListNeedy.most_common(30)

In [ ]:
# Create a list of needy keywords from the most common words
needy_keywords = [word for word, count in importantWordListNeedy]

# Filter the cleaned tweets to get only those that contain the needy keywords
filtered_needy_tweets_freq = {}
for tweetId, tweet in cleaned_needy_tweets.items():
    if any(keyword in tweet.lower() for keyword in needy_keywords):
        filtered_needy_tweets_freq[tweetId] = tweet

**2) Donor**

In [ ]:
# Calculate the frequency distribution of words in the cleaned tweets
frequentWordListDonor = nltk.FreqDist(wordListDonor)
# Get the 30 most common words from the frequency distribution
importantWordListDonor = frequentWordListDonor.most_common(30)

In [ ]:
# Identify donor tweets
donor_keywords = [word for word, count in importantWordListDonor]

# Filter the cleaned tweets to get only those that contain the donor keywords
filtered_donor_tweets_freq = {}
for tweetId, tweet in cleaned_donor_tweets.items():
    if any(keyword in tweet.lower() for keyword in donor_keywords):
        filtered_donor_tweets_freq[tweetId] = tweet

# **Named Entity Recognition**

**1) Needy**

In [ ]:
# Define relevant named entity types
relevant_entities = ["ORG", "PERSON", "MONEY"]

# Create an empty dictionary to store filtered tweets with relevant named entities
filtered_needy_tweets_ner = {}

# Iterate over each tweet and perform NER
for tweetId, tweet in filtered_needy_tweets_freq.items():
    # Create a Doc object of the tweet using spaCy
    doc = nlp(tweet)
    # Extract named entities of relevant types
    entities = [ent for ent in doc.ents if ent.label_ in relevant_entities]
    # If there are no relevant entities in the tweet, skip to the next tweet
    if not entities:
        continue
    # Otherwise, store the tweet in the filtered_needy_tweets_ner dictionary
    filtered_needy_tweets_ner[tweetId] = tweet

**2) Donor**

In [ ]:
# Define relevant named entity types
relevant_entities = ["ORG", "PERSON", "MONEY"]

# Create an empty dictionary to store filtered tweets with relevant named entities
filtered_donor_tweets_ner = {}

# Iterate over each tweet and perform NER
for tweetId, tweet in filtered_donor_tweets_freq.items():
    # Create a Doc object of the tweet using spaCy
    doc = nlp(tweet)
    # Extract named entities of relevant types
    entities = [ent for ent in doc.ents if ent.label_ in relevant_entities]
    # If there are no relevant entities in the tweet, skip to the next tweet
    if not entities:
        continue
    # Otherwise, store the tweet in the filtered_donor_tweets_ner dictionary
    filtered_donor_tweets_ner[tweetId] = tweet

# **POS Tagging**

**1) Needy**

In [ ]:
# Nouns (NN):       "donation", "charity", "help", "support"
# Verbs (VB):       "donate," "give," "support," "help" are frequently used in these tweets.
# Adjectives (JJ):  "generous," "kind," "grateful,"  "thankful" are used to express
#                     gratitude towards donors and supporters.
# Pronouns (PRP):   "we", "us" and "you" are used to refer to the individuals
#                     and organizations involved in the charitable efforts.
# Adverbs (RB):     "kindly," "graciously," and "generously" are used to describe
#                     the manner in which donations and support are given.

relevant_tags = ["NN", "VB", "JJ", "PRP", "RB"]

# Define POS tagging function
def pos_tagging(text):
    tokens = nltk.word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    return tags

# Create empty dictionary to store filtered tweets
filtered_needy_tweets_pos = {}

# Iterate over the tweets that contain relevant entities
for tweetId, tweet in filtered_needy_tweets_ner.items():

    # Perform POS tagging on the tweet text
    tags = pos_tagging(tweet)

    # Check if the tweet contains any relevant POS tags
    if any(tag[1] in relevant_tags for tag in tags):
        filtered_needy_tweets_pos[tweetId] = tweet

**2) Donor**

In [ ]:
# Nouns (NN):       "donation", "charity", "help", "support"
# Verbs (VB):       "donate," "give," "support," "help" are frequently used in these tweets.
# Adjectives (JJ):  "generous," "kind," "grateful,"  "thankful" are used to express
#                     gratitude towards donors and supporters.
# Pronouns (PRP):   "we", "us" and "you" are used to refer to the individuals
#                     and organizations involved in the charitable efforts.
# Adverbs (RB):     "kindly," "graciously," and "generously" are used to describe
#                     the manner in which donations and support are given.

relevant_tags = ["NN", "VB", "JJ", "PRP", "RB"]

# POS tagging function
def pos_tagging(text):
    tokens = nltk.word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    return tags

# Create empty dictionary to store filtered tweets
filtered_donor_tweets_pos = {}

# Iterate over the tweets that contain relevant entities
for tweetId, tweet in filtered_donor_tweets_ner.items():

    # Perform POS tagging on the tweet text
    tags = pos_tagging(tweet)

    # Check if the tweet contains any relevant POS tags
    if any(tag[1] in relevant_tags for tag in tags):
        filtered_donor_tweets_pos[tweetId] = tweet

# **Sentiment Analysis**

**1) Needy**

**a) TextBlob**

In [ ]:
textblob_scores_needy = {}
for tweetId, tweet in filtered_needy_tweets_pos.items():
    # Calculate TextBlob sentiment polarity score for each tweet
    textblob_score = TextBlob(tweet).sentiment.polarity
    # Add the score to the textblob_scores_needy dictionary with tweetId as key
    textblob_scores_needy[tweetId] = textblob_score

**b) Vader**

In [ ]:
vader_scores_needy = {}
for tweetId, tweet in filtered_needy_tweets_pos.items():
    # Initialize the SentimentIntensityAnalyzer
    vader_analyzer = SentimentIntensityAnalyzer()
    # Calculate the Vader sentiment compound score for each tweet
    vader_score = vader_analyzer.polarity_scores(tweet)['compound']
    # Add the score to the vader_scores_needy dictionary with tweetId as key
    vader_scores_needy[tweetId] = vader_score

**c) Flair**

In [ ]:
# Load pre-trained sentiment classifier from Flair
flair_classifier = TextClassifier.load('en-sentiment')

flair_scores_needy = {}
for tweetId, tweet in filtered_needy_tweets_pos.items():
    # Create a Flair Sentence object from the tweet
    flair_sentence = Sentence(tweet)
    # Use the Flair classifier to predict the sentiment of the sentence
    flair_classifier.predict(flair_sentence)
    # flair_scores_needy[tweetId] = flair_sentence.labels[0].score
    # Store the sentiment label with highest confidence score as the sentiment score for the tweet
    flair_scores_needy[tweetId] = flair_sentence.labels[0].value

**Classifying Positive and Negative Tweets:**

In [ ]:
# Initialize dictionaries to store positive and negative tweets based on TextBlob, VADER, and Flair scores

positive_tweets_needy_textblob = {}
negative_tweets_needy_textblob = {}

positive_tweets_needy_vader = {}
negative_tweets_needy_vader = {}

positive_tweets_needy_flair = {}
negative_tweets_needy_flair = {}

# Iterate through each tweet and score them based on their TextBlob, VADER, and Flair sentiment scores
for tweetId, tweet in filtered_needy_tweets_pos.items():
    # Add tweet to the positive dictionary if its TextBlob score is above 0.25, else add it to negative dictionary
    if textblob_scores_needy[tweetId] > 0.25:
        positive_tweets_needy_textblob[tweetId] = tweet
    else:
        negative_tweets_needy_textblob[tweetId] = tweet

    # Add tweet to the positive dictionary if its VADER score is above 0.05, else add it to negative dictionary
    if vader_scores_needy[tweetId] > 0.05:
        positive_tweets_needy_vader[tweetId] = tweet
    else:
        negative_tweets_needy_vader[tweetId] = tweet

    # Add tweet to the positive dictionary if its Flair score is 'POSITIVE', else add it to negative dictionary
    if flair_scores_needy[tweetId] == 'POSITIVE':
        positive_tweets_needy_flair[tweetId] = tweet
    else:
        negative_tweets_needy_flair[tweetId] = tweet

**2) Donor**

**a) TextBlob**

In [ ]:
textblob_scores_donor = {}
for tweetId, tweet in filtered_donor_tweets_pos.items():
    # Calculate TextBlob sentiment polarity score for each tweet
    textblob_score = TextBlob(tweet).sentiment.polarity
    # Add the score to the textblob_scores_donor dictionary with tweetId as key
    textblob_scores_donor[tweetId] = textblob_score

**b) Vader**

In [ ]:
vader_scores_donor = {}
for tweetId, tweet in filtered_donor_tweets_pos.items():
    # Initialize the SentimentIntensityAnalyzer
    vader_analyzer = SentimentIntensityAnalyzer()
    # Calculate the Vader sentiment compound score for each tweet
    vader_score = vader_analyzer.polarity_scores(tweet)['compound']
    # Add the score to the vader_scores_donor dictionary with tweetId as key
    vader_scores_donor[tweetId] = vader_score

**c) Flair**

In [ ]:
# Load pre-trained sentiment classifier from Flair
flair_classifier = TextClassifier.load('en-sentiment')

flair_scores_donor = {}
for tweetId, tweet in filtered_donor_tweets_pos.items():
    # Create a Flair Sentence object from the tweet
    flair_sentence = Sentence(tweet)
    # Use the Flair classifier to predict the sentiment of the sentence
    flair_classifier.predict(flair_sentence)
    # flair_scores_donor[tweetId] = flair_sentence.labels[0].score
    # Store the sentiment label with highest confidence score as the sentiment score for the tweet
    flair_scores_donor[tweetId] = flair_sentence.labels[0].value

**Classifying Positive and Negative Tweets:**

In [ ]:
# Initialize dictionaries to store positive and negative tweets based on TextBlob, VADER, and Flair scores

positive_tweets_donor_textblob = {}
negative_tweets_donor_textblob = {}

positive_tweets_donor_vader = {}
negative_tweets_donor_vader = {}

positive_tweets_donor_flair = {}
negative_tweets_donor_flair = {}

# Iterate through each tweet and score them based on their TextBlob, VADER, and Flair sentiment scores
for tweetId, tweet in filtered_donor_tweets_pos.items():
    # Add tweet to the positive dictionary if its TextBlob score is above 0.25, else add it to negative dictionary
    if textblob_scores_donor[tweetId] > 0.25:
        positive_tweets_donor_textblob[tweetId] = tweet
    else:
        negative_tweets_donor_textblob[tweetId] = tweet

    # Add tweet to the positive dictionary if its VADER score is above 0.05, else add it to negative dictionary
    if vader_scores_donor[tweetId] > 0.05:
        positive_tweets_donor_vader[tweetId] = tweet
    else:
        negative_tweets_donor_vader[tweetId] = tweet

    # Add tweet to the positive dictionary if its Flair score is 'POSITIVE', else add it to negative dictionary
    if flair_scores_donor[tweetId] == 'POSITIVE':
        positive_tweets_donor_flair[tweetId] = tweet
    else:
        negative_tweets_donor_flair[tweetId] = tweet

# **Metrics**

**a) Bar Graphs**

**Positive Tweets of Needy & (Volunteer/Donor) Groups for TextBlob, Vader, Flair:**

In [ ]:
import matplotlib.pyplot as plt

# Plot Textblob sentiment analysis results
fig, ax = plt.subplots()
ax.bar(['Needy', 'Volunteer'], [len(positive_tweets_needy_textblob), len(positive_tweets_donor_textblob)])
ax.set_ylabel('Number of Tweets')
ax.set_title('Textblob - Twitter Analysis Results')
plt.show()

# Plot Vader sentiment analysis results
fig, ax = plt.subplots()
ax.bar(['Needy', 'Volunteer'], [len(positive_tweets_needy_vader), len(positive_tweets_donor_vader)])
ax.set_ylabel('Number of Tweets')
ax.set_title('Vader - Twitter Analysis Results')
plt.show()

# Plot Flair sentiment analysis results
fig, ax = plt.subplots()
ax.bar(['Needy', 'Volunteer'], [len(positive_tweets_needy_flair), len(positive_tweets_donor_flair)])
ax.set_ylabel('Number of Tweets')
ax.set_title('Flair - Twitter Analysis Results')
plt.show()



**b) Pie Charts**

**1) Sentiment Distribution of Needy**

In [ ]:
# Text Blob - Needy Pie Chart Sentiment
totalNeedy_Textblob = len(positive_tweets_needy_textblob) + len(negative_tweets_needy_textblob)
labels = ['Positive', 'Negative']
sizes = [format(100 * len(positive_tweets_needy_textblob) / totalNeedy_Textblob), format(100 * len(negative_tweets_needy_textblob) / totalNeedy_Textblob)]
colors = ['green', 'red']
# Create a pie chart
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
# Add a title
plt.title('TextBlob - Needy Sentiment distribution')
# Show the chart
plt.show()

print()

# Vader - Needy Pie Chart Sentiment
totalNeedy_Vader = len(positive_tweets_needy_vader) + len(negative_tweets_needy_vader)
labels = ['Positive', 'Negative']
sizes = [format(100 * len(positive_tweets_needy_vader) / totalNeedy_Vader), format(100 * len(negative_tweets_needy_vader) / totalNeedy_Vader)]
colors = ['green', 'red']
# Create a pie chart
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
# Add a title
plt.title('Vader - Needy Sentiment distribution')
# Show the chart
plt.show()

print()

# Flair - Needy Pie Chart Sentiment
totalNeedy_Flair = len(positive_tweets_needy_flair) + len(negative_tweets_needy_flair)
labels = ['Positive', 'Negative']
sizes = [format(100 * len(positive_tweets_needy_flair) / totalNeedy_Flair), format(100 * len(negative_tweets_needy_flair) / totalNeedy_Flair)]
colors = ['green', 'red']
# Create a pie chart
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
# Add a title
plt.title('Flair - Needy Sentiment distribution')
# Show the chart
plt.show()

**2) Sentiment Distribution of Donors**

In [ ]:
# Text Blob - Donor Pie Chart Sentiment
totalDonor_Textblob = len(positive_tweets_donor_textblob) + len(negative_tweets_donor_textblob)
labels = ['Positive', 'Negative']
sizes = [format(100 * len(positive_tweets_donor_textblob) / totalDonor_Textblob), format(100 * len(negative_tweets_donor_textblob) / totalDonor_Textblob)]
colors = ['green', 'red']
# Create a pie chart
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
# Add a title
plt.title('TextBlob - Donor Sentiment distribution')
# Show the chart
plt.show()

print()

# Vader - Donor Pie Chart Sentiment
totalDonor_Vader = len(positive_tweets_donor_vader) + len(negative_tweets_donor_vader)
labels = ['Positive', 'Negative']
sizes = [format(100 * len(positive_tweets_donor_vader) / totalDonor_Vader), format(100 * len(negative_tweets_donor_vader) / totalDonor_Vader)]
colors = ['green', 'red']
# Create a pie chart
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
# Add a title
plt.title('Vader - Donor Sentiment distribution')
# Show the chart
plt.show()

print()

# Flair - Donor Pie Chart Sentiment
totalDonor_Flair = len(positive_tweets_donor_flair) + len(negative_tweets_donor_flair)
labels = ['Positive', 'Negative']
sizes = [format(100 * len(positive_tweets_donor_flair) / totalDonor_Flair), format(100 * len(negative_tweets_donor_flair) / totalDonor_Flair)]
colors = ['green', 'red']
# Create a pie chart
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
# Add a title
plt.title('Flair - Donor Sentiment distribution')
# Show the chart
plt.show()

**c) Choropleth Map**

**1) Choropleth Map of Needy in USA**

In [ ]:
# List of all US States
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

# List of all US state codes
statesShortForm = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# For Needy - Flair
# Create a defaultdict to store the count of needy tweets for each state
stateNeedyCount = defaultdict(int)

# Iterate over each needy tweet in positive_tweets_needy_vader
for tweetId, tweet in positive_tweets_needy_vader.items():

  # Get the location information from the tweet's data in needyTweets dictionary
  loc = needyTweets[tweetId]['location']

  # Check if the location string contains the name or state code for any US state
  for i in range(50):
    if states[i] in loc or statesShortForm[i] in loc:
      # If the tweet belongs to a US state, increment the count for that state
      stateNeedyCount[statesShortForm[i]] += 1
      break

# Create two empty lists to store the state codes and their respective counts
states, countsvalue = [], []

# Iterate over the stateNeedyCount defaultdict and store the keys (state codes) and values (count of needy tweets) in the above lists
for key, value in stateNeedyCount.items():
  states.append(key)
  countsvalue.append(value)

# Define the choropleth plot's data and layout parameters
data = dict(type = 'choropleth',
                locations = states,
                locationmode = 'USA-states',
                z = countsvalue,
                colorscale = 'Greens',
                colorbar = {'title' : 'colorbar'})
layout = dict(title = 'Needy in USA',
                geo = dict(scope='usa' ,
                showlakes = True,
                lakecolor = 'rgb(0,191,255)'))

# Create a new figure object using the plotly.graph_objs module
x = pg.Figure(data = [data], layout = layout)

# Use the plotly.offline.iplot() method to plot the choropleth figure
po.iplot(x)

**2) Choropleth Map of Donors in USA**

In [ ]:
# List of all US States
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

# List of all US state codes
statesShortForm = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# For Donor - Flair
stateDonorCount = defaultdict(int)

# Iterate over each donor tweet in positive_tweets_donor_vader
for tweetId, tweet in positive_tweets_donor_vader.items():

  # Get the location information from the tweet's data in needyTweets dictionary
  loc = donorTweets[tweetId]['location']

  # Check if the location string contains the name or state code for any US state
  for i in range(50):
    if states[i] in loc or statesShortForm[i] in loc:
      # If the tweet belongs to a US state, increment the count for that state
      stateDonorCount[statesShortForm[i]] += 1
      break

# Create two empty lists to store the state codes and their respective counts
states, countsvalue = [], []

# Iterate over the stateDonorCount defaultdict and store the keys (state codes) and values (count of donor tweets) in the above lists
for key, value in stateDonorCount.items():
  states.append(key)
  countsvalue.append(value)

# Define the choropleth plot's data and layout parameters
data = dict(type = 'choropleth',
                locations = states,
                locationmode = 'USA-states',
                z = countsvalue,
                colorscale = 'Greens',
                colorbar = {'title' : 'colorbar'})
layout = dict(title = 'Volunteers in USA',
                geo = dict(scope='usa' ,
                showlakes = True,
                lakecolor = 'rgb(0,191,255)'))

# Create a new figure object using the plotly.graph_objs module
x = pg.Figure(data = [data], layout = layout)

# Use the plotly.offline.iplot() method to plot the choropleth figure
po.iplot(x)

# **Results**

**1) Needy**

In [ ]:
# Create an empty list to store information about needy tweets.
needyTweetList = []

# Iterate over each needy tweet
for tweetId, tweet in positive_tweets_needy_flair.items():
  # Extract the full text of the tweet using its tweet ID from the needyTweets dictionary and append it as a key-value pair to the needyTweetList list.
  needyTweetList.append({
      'Needy Tweets': needyTweets[tweetId]['full_text']
  })

# Create a pandas DataFrame from the extracted information
df = pd.DataFrame(needyTweetList)

# Save the DataFrame to a CSV file
df.to_csv("Needy Tweet Info.csv", index=False)

# Download the CSV file from the colab notebook.
files.download('Needy Tweet Info.csv')

**2) Donors**

In [ ]:
# Create an empty list to store information about donor tweets.
donarTweetList = []

# Iterate over each donor tweet
for tweetId, tweet in positive_tweets_donor_flair.items():
  # Extract the full text of the tweet using its tweet ID from the donorTweets dictionary and append it as a key-value pair to the donarTweetList list.
  donarTweetList.append({
      'Donor Tweets': donorTweets[tweetId]['full_text']
  })

# Create a pandas DataFrame from the extracted information
df = pd.DataFrame(donarTweetList)

# Save the DataFrame to a CSV file
df.to_csv("Volunteer Tweet Info.csv", index=False)

# Download the CSV file from the colab notebook.
files.download('Volunteer Tweet Info.csv')